In [3]:
import ee
from geemap import Map

ee.Initialize()

In [4]:
# Define AOI and feature collections
aoi = ee.FeatureCollection("projects/ee-classes24/assets/Narok")
crop = ee.FeatureCollection("projects/ee-classes24/assets/crop_narok")
other = ee.FeatureCollection("projects/ee-classes24/assets/other_narok")

In [5]:
# Load Sentinel-2 ImageCollection and filter by AOI
sentinel2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterBounds(aoi)

# Define date ranges for bi-weekly periods
date_ranges = [
    ['2023-03-01', '2023-03-15'], ['2023-03-16', '2023-03-31'],
    ['2023-04-01', '2023-04-15'], ['2023-04-16', '2023-04-30'],
    ['2023-05-01', '2023-05-15'], ['2023-05-16', '2023-05-31'],
    ['2023-06-01', '2023-06-15'], ['2023-06-16', '2023-06-30'],
    ['2023-07-01', '2023-07-15'], ['2023-07-16', '2023-07-31'],
    ['2023-08-01', '2023-08-15'], ['2023-08-16', '2023-08-31']
]

In [6]:
# Cloud masking function
def mask_s2_clouds(image):
    shadow = image.select('SCL').eq(3)  # Cloud shadows
    cirrus = image.select('SCL').eq(10)  # Cirrus clouds
    mask = image.select('QA60').Not().And(shadow.Not()).And(cirrus.Not())
    return image.updateMask(mask).copyProperties(image, ["system:time_start"])

# Function to compute NDVI and greenest pixel composite
def compute_greenest_composite(date_range):
    filtered = sentinel2.filterDate(date_range[0], date_range[1])
    filtered = filtered.map(mask_s2_clouds)

    def add_ndvi(img):
        ndvi = img.expression(
            '(B8 - B4) / (B8 + B4)', {
                'B8': img.select('B8'),
                'B4': img.select('B4')
            }
        ).rename('NDVI')
        return img.addBands(ndvi)

    with_ndvi = filtered.map(add_ndvi)
    mosaic = with_ndvi.qualityMosaic('NDVI')
    return mosaic.addBands(mosaic.select('NDVI')).select(['B2', 'B3', 'B4', 'B5', 'B8', 'B11', 'B12', 'NDVI']).clip(aoi)

In [7]:
# Compute greenest composites for each date range
greenest_composites = [compute_greenest_composite(range) for range in date_ranges]

# Extract NDVI bands and stack them
def extract_ndvi(img, index):
    return img.select('NDVI').rename(f'NDVI{index + 1}')

ndvi_bands = [extract_ndvi(img, i) for i, img in enumerate(greenest_composites)]
stacked_ndvi = ee.ImageCollection(ndvi_bands).toBands().multiply(100).add(100).uint8()

# Merge crop and other feature collections
training_features = crop.merge(other)

# Sample training data
training_data = stacked_ndvi.sampleRegions(
    collection=training_features,
    properties=['class'],
    scale=30
)


In [9]:
# Train classifier
classifier = ee.Classifier.smileRandomForest(100).train(
    features=training_data,
    classProperty='class'
)

# Classify the stacked NDVI
classified = stacked_ndvi.classify(classifier, 'Classified')

# Mask for class 1 and visualize
classified1 = classified.updateMask(classified.eq(1))

# Visualize using geemap
m = Map()
m.add_basemap('HYBRID')  # Set background to Google Satellite Hybrid
m.centerObject(aoi, 10)  # Zoom to AOI
m.addLayer(stacked_ndvi, {'bands': ['8_NDVI9', '5_NDVI6', '2_NDVI3'], 'min': 0, 'max': 200}, 'Stacked_NDVI')
m.addLayer(greenest_composites[5], {'bands': ['B8', 'B4', 'B3'], 'min': 0, 'max': 4000}, 'Greenest_Composite')
m.addLayer(classified1, {'palette': ['yellow'], 'min': 0, 'max': 1}, 'Classified')
m

Map(center=[-1.2534642587579428, 35.575071087965014], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
# Export the classified image
export_task = ee.batch.Export.image.toDrive(
    image=classified1.clip(aoi),
    description='classified_image',
    folder='GEE_exports',
    region=aoi.geometry().bounds().getInfo()['coordinates'],
    scale=30,
    crs='EPSG:4326'
)
export_task.start()